In [3]:
from dotenv import load_dotenv
import netCDF4 as nc
import numpy as np
import os
import requests
from terracatalogueclient import Catalogue
from terracatalogueclient.config import CatalogueConfig, CatalogueEnvironment
import pandas as pd
import urllib.request
from urllib.error import HTTPError
import xarray as xr

In [8]:
# Carga las variables de entorno desde el archivo .env
load_dotenv()

# Accede a la contraseña desde las variables de entorno
password = os.getenv('COPERNICUS_PASSWORD')
username = os.getenv('COPERNICUS_USERNAME')

In [4]:

def download_nc_file_with_auth(url, output_file, username, password):
    # Create a password manager with the given username and password
    password_mgr = urllib.request.HTTPPasswordMgrWithDefaultRealm()
    password_mgr.add_password(None, url, username, password)
    handler = urllib.request.HTTPBasicAuthHandler(password_mgr)
    opener = urllib.request.build_opener(handler)
    
    try:
        # Open the URL with the configured opener
        response = opener.open(url)
        
        with open(output_file, 'wb') as f:
            f.write(response.read())
        
        print(f"Downloaded {url} to {output_file}")
    except HTTPError as e:
        print(f"Failed to download {url}. Status code: {e.code}, Reason: {e.reason}")



def slice_ds(lat_min, lat_max, lon_min, lon_max, ds):

    lat_min_ds = ds.lat.values.min()
    lat_max_ds = ds.lat.values.max()
    lat_len_ds = len(ds.lat.values)

    lon_min_ds = ds.lon.values.min()
    lon_max_ds = ds.lon.values.max()
    lon_len_ds = len(ds.lon.values)

    lat_coord = np.linspace(lat_max_ds, lat_min_ds, lat_len_ds)
    lon_coord = np.linspace(lon_min_ds, lon_max_ds, lon_len_ds)

    new_lat_min = np.absolute(lat_coord-lat_min).argmin()
    new_lat_max = np.absolute(lat_coord-lat_max).argmin()
    new_lon_min = np.absolute(lon_coord-lon_min).argmin()
    new_lon_max = np.absolute(lon_coord-lon_max).argmin()

    return ds.NDVI[0, new_lat_max:new_lat_min, new_lon_min:new_lon_max]

In [33]:
link_1 = f"https://land.copernicus.vgt.vito.be/manifest/ndvi_v3_1km/manifest_cgls_ndvi_v3_1km_latest.txt" #2020 - 1999
link_2 = f"https://land.copernicus.vgt.vito.be/manifest/ssm_v1_1km/manifest_cgls_ssm_v1_1km_latest.txt" #SSM 2014
link_3 = f"https://land.copernicus.vgt.vito.be/manifest/fapar_v2_1km/manifest_cgls_fapar_v2_1km_latest.txt" #FAPAR 1999 - 202006

link_4 = f"https://land.copernicus.vgt.vito.be/PDF/datapool/Vegetation/Indicators/NDVI_1km_V2_LTS/1999/12/21/NDVI-LTS_1999-2017-1221_GLOBE_VGT-PROBAV_V2.2.1/c_gls_NDVI-LTS_1999-2017-1221_GLOBE_VGT-PROBAV_V2.2.1.nc"

links = [link_1, link_2, link_3]
folders = ["nvdi", "ssm", "fapar"]

link_dict = {link:folder for link, folder in zip(links, folders)}

main_data_folder = f"../data"

In [35]:


for link, folder in link_dict.items():

    data_folder = f"{main_data_folder}/{folder}"
    if os.path.exists(data_folder) is False:
        os.mkdir(data_folder)

    response = requests.get(link)

    response.raise_for_status()

    nc_files = response.text.split("\n")

    nc_file = nc_files[0]

    #for nc_file in nc_files:

    filename = nc_file.split("/")[-1]

    if ".nc" in filename:
        output_filename = f'{data_folder}/{filename}'
        download_nc_file_with_auth(nc_file, output_filename, username, password)

        ds = xr.open_dataset(output_filename)

        ds_slice = slice_ds(lat_min=36, lat_max=46, lon_min=-10, lon_max=3, ds=ds)


        ds_slice.to_netcdf(output_filename)

        ds.close()
        ds_slice.to_netcdf(f"{data_folder}/sample.nc")

    break





Downloaded https://land.copernicus.vgt.vito.be/PDF/datapool/Vegetation/Indicators/NDVI_1km_V3/2020/06/21/NDVI_202006210000_GLOBE_PROBAV_V3.0.1/c_gls_NDVI_202006210000_GLOBE_PROBAV_V3.0.1.nc to ../data/nvdi/c_gls_NDVI_202006210000_GLOBE_PROBAV_V3.0.1.nc


PermissionError: [Errno 13] Permission denied: '/home/carlossi/zrive/zrive-ds-4q23-olive/data/nvdi/c_gls_NDVI_202006210000_GLOBE_PROBAV_V3.0.1.nc'

In [15]:
config = CatalogueConfig.from_environment(CatalogueEnvironment.CGLS)
catalogue = Catalogue(config)

In [59]:
import pandas as pd
from itertools import chain
collections = catalogue.get_collections()

rows = []
for c in collections:
    values_list = list(c.properties.values())
    values_list.insert(0, c.id)
    rows.append(values_list)

column_values = list(c.properties.keys())
column_values.insert(0, "Identifier")

df = pd.DataFrame(data = rows, columns = column_values)
# df.style.set_properties(**{'text-align': 'left'})


In [16]:
collections = catalogue.get_collections()
rows = []
for c in collections:
    rows.append([c.id, c.properties['title']])

df_catalogue = pd.DataFrame(data = rows, columns = ['Identifier', 'Description'])
# df_catalogue.to_csv("catalogue_short.txt", sep="\t")

df_catalogue.head()

,Identifier,Description
0,clms_global_albh_1km_v1_10daily_netcdf,Broadband Hemispherical Surface Albedo: global...
1,clms_global_aldh_1km_v1_10daily_netcdf,Broadband Hemispherical Surface Albedo: global...
2,clms_global_ba_300m_v3_daily_netcdf,Burnt Area: global daily (raster 300m)
3,clms_global_ba_300m_v3_monthly_netcdf,Burnt Area: global monthly (raster 300m)
4,clms_global_dmp_1km_v2_10daily_netcdf,Dry Matter Productivity: global 10-daily (rast...


In [18]:
import pandas as pd
import datetime as dt

rows = []
products = catalogue.get_products(
    "clms_global_ndvi_1km_v2_10daily_netcdf",
    start=dt.date(2019, 9, 1),
    end=dt.date(2019, 9, 1),
    bbox = [-10, 36, 3, 46]
)

for product in products:
    rows.append([product.id, product.data[0].href, (product.data[0].length/(1024*1024))])

df = pd.DataFrame(data = rows, columns = ['Identifier', 'URL', 'Size (MB)'])
df.head()

,Identifier,URL,Size (MB)
0,c_gls_NDVI_201909010000_GLOBE_PROBAV_V2.2.1,https://globalland.vito.be/download/netcdf/ndv...,201.317039


In [32]:
product_list = list(catalogue.get_products(
    "clms_global_ndvi_1km_v2_10daily_netcdf",
    start=dt.date(2019, 9, 1),
    end=dt.date(2019, 9, 3),
    bbox = [-10, 36, 3, 46]
))

# catalogue.download_products(product_list, './')

In [38]:
a = catalogue.get_products(
    "clms_global_ndvi_1km_v2_10daily_netcdf",
    start=dt.date(2019, 9, 1),
    end=dt.date(2019, 9, 3),
    bbox = [-100, -80, 3, 46]

)

for i in a:
    print(i.bbox)

[-180.0044642857, -59.9955357143, 179.9955357143, 80.0044642857]


In [43]:
import datetime
import requests
import matplotlib.pyplot as plt
import numpy as np
tsvBaseURL='https://services.terrascope.be/catalogue/'

response = requests.get(tsvBaseURL) #this returns the layers that are available

In [46]:
Catalogdescription=requests.get(tsvBaseURL+'description')
CatalogJson=Catalogdescription.json()

In [48]:
pd.json_normalize(CatalogJson)

,esip_version,ShortName,LongName,Description,Url,Query,Tags,Developer,Attribution,SyndicationRight,AdultContent,Language,OutputEncoding,InputEncoding,esipVersion
0,1.2,VITO EO Catalog,New VITO Earth Observation Catalogue,"Provides interoperable access, following ISO/O...",[{'template': 'https://services.terrascope.be/...,"[{'role': 'example', 'start': '2015-01-01T12:3...",VITO EarthObservation Terrascope Catalogue,VITO,"Copyright 2023, VITO",OPEN,False,en-us,UTF-8,UTF-8,1.2


In [49]:
CatalogJson

{'esip_version': '1.2',
 'ShortName': 'VITO EO Catalog',
 'LongName': 'New VITO Earth Observation Catalogue',
 'Description': 'Provides interoperable access, following ISO/OGC interface guidelines, to Earth Observation metadata.',
 'Url': [{'template': 'https://services.terrascope.be/catalogue/description',
   'type': 'application/geo+json',
   'rel': 'self'},
  {'template': 'https://services.terrascope.be/catalogue/collections.geojson?httpAccept={sru:httpAccept?}&recordSchema={sru:recordSchema?}&count={os:count?}&startIndex={os:startIndex?}&startPage={os:startPage?}&q={os:searchTerms?}&title={eo:title?}&denominator={eo:denominator?}&distanceValue={eo:distanceValue?}&distanceUOM={eo:distanceUOM?}&resolution={eo:resolution?}&specificationTitle={eo:specificationTitle?}&specificationDate={eo:specificationDate?}&specificationdateType={eo:specificationdateType?}&degree={eo:degree?}&topicCategory={eo:topicCategory?}&keyword={eo:keyword?}&abstract={eo:abstract?}&organisationName={eo:organisat

In [44]:
if response.status_code == 200:
    layerlist = response.json()['layers']
else:
    raise IOError(response.text)

OSError: {
    "@context": "http://schemas.opengis.net/os-geojson/1.0/os-geojson-11.jsonld",
	"type": "ExceptionReport",
	"exceptions": [
		{  "type": "Exception",
			"exceptionCode": "http://www.opengis.net/ows/2.0#NoApplicableCode",
			"exceptionText": "The requested operation is not available"
		}
	]
}

In [41]:
layersSorted = sorted(layerlist, key=lambda k: k['name']) 


In [42]:
count=0
for l in layersSorted:
    if len(l['dates'])>0:
        count=count+1
        print(l['name'].ljust(40), '{0:4d} '.format(len(l['dates'])), min(l['dates'])[:19], 'to', max(l['dates'])[:19])
print(">>>", count, 'layers with data')
print('Layers with no dates')
count=0
for l in layersSorted:
    if len(l['dates'])==0:
        count=count+1
        print(l['name'])
print(">>>", count, 'layers without data')

BCGMS_DMP300                              348  2014-01-10T00:00:00 to 2023-09-30T00:00:00
BCGMS_DMP300_MASKED                       298  2014-01-10T00:00:00 to 2022-04-10T00:00:00
BCGMS_FAPAR300                            348  2014-01-10T00:00:00 to 2023-09-30T00:00:00
BCGMS_FAPAR300_MASKED                     296  2014-01-10T00:00:00 to 2022-04-10T00:00:00
BCGMS_NDVI300                             253  2014-01-01T00:00:00 to 2021-01-01T00:00:00
BCGMS_NDVI300_MASKED                      253  2014-01-01T00:00:00 to 2021-01-01T00:00:00
BIOPAR_FAPAR300_V1_GLOBAL                 816  2014-01-10T00:00:00 to 2023-11-10T00:00:00
BIOPAR_FAPAR_V2_GLOBAL                    786  1998-09-10T00:00:00 to 2020-06-30T00:00:00
BIOPAR_LAI300_V1_GLOBAL                   812  2014-01-10T00:00:00 to 2023-11-10T00:00:00
BIOPAR_LAI_V2_GLOBAL                      786  1998-09-10T00:00:00 to 2020-06-30T00:00:00
BIOPAR_NDVI300_V2_GLOBAL                  121  2020-07-01T00:00:00 to 2023-11-01T00:00:00
BIOPAR_NDV

In [ ]:
list = ["BIOPAR_NDVI_V2_GLOBAL",
        ]